In [1]:
import requests
from time import sleep

auth_key = '60ae8437555642ea85b9a1ab42bad9fe'

# store global constants
headers = {
   "authorization": auth_key, 
   "content-type": "application/json"
}
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'

# make a function to pass the mp3 to the upload endpoint
def read_file(filename):
   with open(filename, 'rb') as _file:
       while True:
           data = _file.read(5242880)
           if not data:
               break
           yield data

# upload our audio file
upload_response = requests.post(
   upload_endpoint,
   headers=headers, data=read_file("talk.mp3") #add your video file here 
)
# send a request to transcribe the audio file
transcript_request = {'audio_url': upload_response.json()['upload_url']}
transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
# set up polling
polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
filename = transcript_response.json()['id'] + '.txt'
# if our status isn’t complete, sleep and then poll again
while polling_response.json()['status'] != 'completed':
   sleep(30)
   polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
   print("File is", polling_response.json()['status'])
with open(filename, 'w') as f:
   f.write(polling_response.json()['text'])
print('Transcript saved to', filename)

File is processing
File is processing
File is processing
File is processing
File is processing
File is processing
File is completed
Transcript saved to rzq4l0wtqx-28fa-4844-84e0-e99c6ddc0fe8.txt
